<center><h1>Introduction to Equistore</h1></center>

This document present [equistore](https://github.com/lab-cosmo/equistore), a
format for data exchange between atomistic machine learning software. We will go
over the basics of this format, and how one can exploit the metadata to simplify
building complex models.

You can install all the dependency used in this tutorial with pip:

```
pip install -r requirements.txt
```

# Exploring an equivariant representation: the spherical expansion

One of the simplest equivariant representation is the SOAP spherical expansion. It is built by representing atoms as multiple atomic density (one per species $\alpha$) $\braket{r|\rho_i^\alpha}$, and then expanding this density on a set of radial and angular basis functions: $\braket{n l m |\rho_i^\alpha}$. 

See the ["Efficient implementation of atom-density representations"](https://aip.scitation.org/doi/10.1063/5.0044689) article or the ["Physics-Inspired Structural Representations for Molecules and Materials"](https://pubs.acs.org/doi/10.1021/acs.chemrev.1c00021) review for more information on the
spherical expansion and SOAP.

In this tutorial, we will use [rascaline](https://github.com/Luthaf/rascaline/)
to compute the spherical expansion. Rascaline is a prototype library to compute
SOAP and related representations directly using the equistore format, but is not
required at all when using equistore!

On the contrary, one of the explicit goal of equistore is to be useable as a
standalone tool, and integrate well with different libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import rascaline
import equistore
import equistore.io

import ase.io

We will use a small dataset of water molecules, containing information about the energy and forces of each structure

In [ ]:
structures = ase.io.read("data/water-dipoles.xyz", ":10")

Let's use rascaline to compute the spherical expansion, and get the
representation of all these structures

In [ ]:
# see https://luthaf.fr/rascaline/ for the documentation of rascaline
calculator = rascaline.SphericalExpansion(
    cutoff=3.0, 
    max_radial=5, 
    max_angular=6, 
    atomic_gaussian_width=0.3, 
    radial_basis={"Gto": {}}, 
    center_atom_weight=1.0, 
    gradients=True, 
    cutoff_function={"ShiftedCosine": {"width": 0.5}}
)

representation = calculator.compute(structures)

`representation` is now a TensorMap, the core object in equistore

In [ ]:
representation

A TensorMap is similar to a dictionary: it contains multiple keys, and a block of data is associated with each key

In [ ]:
representation.keys

The keys (like all metadata in equistore) are represented as a set of `Labels`.

A `Labels` instance contains multiple entries, each entry containing multiple variables. 
The variables names define what is being represented. 

Here, we have three variables in the keys: `species_center` and
`species_neighbor` define the species of the central atom (the spherical
expansion is an atom-centered representation) and neighbor atomic density. These
two variables are used to exploit the inherent sparsity of atom-density
representations.

`spherical_harmonics_l` defines which set of spherical harmonics is being used
in the corresponding data block, carrying information about the equivariance of
the representation.

In [ ]:
print(representation.keys.names)

In [ ]:
# two different entries in the keys, corresponding to two different blocks
print(representation.keys[0])
print(representation.keys[18])

One can also access `Labels` one (or multiple) variable at the time, which is useful when transforming the data

In [ ]:
print("spherical_harmonics_l =", representation.keys["spherical_harmonics_l"])
print("\n\nspecies =", representation.keys[["species_center", "species_neighbor"]])

We can get the blocks associated with a given key (or set of keys) in multiple
ways. The most basic is to get the block by the index of the associated key:

In [ ]:
block = representation.block(18)
block

Alternatively, one can get a block by explicitly selecting some of the key variables:

In [ ]:
block = representation.block(spherical_harmonics_l=3, species_center=8, species_neighbor=1)
block

And even get multiple blocks by only giving a subset of the variables:

In [ ]:
blocks = representation.block(spherical_harmonics_l=3)

print(type(blocks), "containing", len(blocks), "blocks")

Blocks as represented by the `TensorBlock` class. Each block contains a main set
of values (the representation itself), metadata associated with these values, and 
optionally gradients of the values with respect to multiple parameters.

In [ ]:
block = representation.block(spherical_harmonics_l=3, species_center=1, species_neighbor=8)

print(block.values.dtype)
print(block.values.shape)

# uncomment this to see the raw representation
# print(block.values)

The values are stored in an n-dimensional array, with at least 2 dimensions. The
first dimension correspond to the **samples** (also sometimes called inputs or
batches), i.e. **what** is being represented. 

The samples are describe by a set of `Labels`, with one entry in the labels for
each row of the array. 

For the spherical expansion, the samples contain information about which atom in
which structure each row corresponds to.

In [ ]:
block.samples

The last dimension of the values array correspond to the **properties**, i.e.
**how** we are representing something, or actual physical properties.

They are also described by a set of `Labels`. For the spherical expansion there
is only one property: the index $n$ of the radial basis.

In [ ]:
block.properties

Any additional dimensions (dimensions between $1$ and $n - 1$) is used to define
**components**. Components are used to represent any kind of vectorial or
tensorial components. This is where we will find metadata related to
equivariance.

Another way of thinking about the difference between components and
samples/features is that one can sub-select samples or properties (if we are
only interested in a subset of the data), but we always need to keep all of the
components entries.

Once again, components are represented as a list of `Labels`. For the spherical
expansion values, there is a single component: the spherical harmonic $m$
number, running from $-l$ to $l$:

In [ ]:
print(type(block.components))
print(f"this block contains {len(block.components)} components")

In [ ]:
block.components[0]

Overall, the block we are currently looking at contains 20 samples (2 hydrogen
per structure, and 10 structure), 7 `spherical_harmonics_m` components (since
`spherical_harmonics_m=3`), and 5 properties (since we set `max_radial=5` in the
calculator).

In [ ]:
print(block.values.shape)

## Gradient data

Each block can also contain the gradients of the values with respect to some
parameters. 

Here, we have the gradient of the representation with respect to the positions
of the atoms.

In [ ]:
gradient = block.gradient("positions")

In the same way that the block contains both data and metadata, the gradients
contains both numeric data and associated metadata

In [ ]:
print(gradient.data.shape)

print(gradient.properties)

The properties are the same between the values and gradients, but the components
and samples can differ. Here we have one additional component describing the
cartesian direction in which we are taking the gradient (x/y/z)

In [ ]:
print(f"we have {len(gradient.components)} components")

gradient.components[0]

And we have the `spherical_harmonics_m` component again

In [ ]:
gradient.components[1]

The gradient samples contains three variables: `sample`, `structure` and `atom`. 

For each row in the gradient data, they indicate which one of the row in the
values we are taking the gradient of, and with respect to which atom in which
structure we are taking this gradient.

In [ ]:
gradient.samples

Here, we are looking at a block with `species_center=1, species_neighbor=8`. 

The first sample in the values is associated with the first H in the first
structure, and there are two gradient samples associated with `sample = 0`. One
of them contains the gradient of the representation with respect to the
positions of atom 0 (the oxygen atom); and the second contains the gradient of
with respect to the positions of atom 1 (the current center).

The next value sample is describing the second hydrogen atom (index 2) in the
structure. The associated gradient samples are very similar, containing
gradients with respect to the position of the oxygen and the atomic center.

In [ ]:
print(block.samples.names)
print(block.samples)

gradient.samples[:4]

## Going from a sparse to a dense representation

Being able to exploit inherent species sparsity in the data is good, but can
make the code using the corresponding code more complex. Fortunately, there are
ways to transform a TensorMap from a sparse to to a dense representation by
merging blocks that share non-sparse keys together.

We'll start fresh with a new spherical expansion representation, which contains 
a lot of separate keys/blocks:

In [ ]:
representation = calculator.compute(structures)

representation.keys

The first function converting from sparse to dense representation is
`keys_to_properties`. This function takes as input the name of a variable from
the keys, and group together blocks that have the same value for all the other
variables in the keys. If we move `species_neighbor` to the properties, all
blocks with the same `spherical_harmonics_l` and `species_center` will be merged
together.

In [ ]:
representation.keys_to_properties("species_neighbor", sort_samples=True)

representation.keys

The merged blocks properties now contain two variables: the species of the
neighbor atoms and the already existing radial basis index $n$.


| species_neighbor=1          | species_neighbor=8          |
|-----------------------------|-----------------------------|
| n=0 / n=1 / n=2 / n=3 / n=4 | n=0 / n=1 / n=2 / n=3 / n=4 |

In [ ]:
representation.block(0).properties

The second function is `keys_to_samples`. It behaves similarly to
`keys_to_properties` regarding which blocks it merges together, but the blocks
are merged along the sample dimension instead of the property dimension:

In [ ]:
representation.keys_to_samples("species_center", sort_samples=True)

representation.keys

Here the samples have been modified (and re-sorted) to contain information about the species of the center.

In [ ]:
representation.block(0).samples

Now the different blocks can be used directly and passed to all the usual ML libraries

In [ ]:
block_l_1 = representation.block(spherical_harmonics_l=1)

block_l_1.values.shape

# Saving data to the disk

TensorMap can be serialized and saved to the disk, using a format based on 
numpy NPZ files.

In [ ]:
equistore.io.save("spherical-expansion.npz", representation)

They can then be loaded again with equistore. 


This can be very useful when computing a part of the machine learning pipeline
is very costly, for example computing a large kernel only once and then slicing
it to create a learning curve, or using complex structural representations

In [ ]:
representation = equistore.io.load("spherical-expansion.npz")

The file format is fully compatible with numpy, and does not requires equistore
to extract the data.

In [ ]:
numpy_loaded = np.load("spherical-expansion.npz")
numpy_loaded["keys"] == representation.keys

# Building ML models

Let's now use equistore to build a simple ML model. Here we will not exploit any
of the equivariant features in equistore, but we will use the metadata
associated with each block.

In [ ]:
from equistore import Labels, TensorBlock, TensorMap

We will start by reading the full water dataset, and then splitting it into
test and train set.

In [ ]:
import sklearn.model_selection

structures = ase.io.read("data/water-dipoles.xyz", ":")
train_structures, test_structures = sklearn.model_selection.train_test_split(
    structures,
    test_size=0.2,
)

Then, we will extract the energy and forces from the dataset, and put them into 
an equistore TensorMap

In [ ]:
def extract_energy_forces(structures):
    energies = []
    forces = []
    for structure in structures:
        energies.append(structure.info["energy"])
        forces.append(structure.arrays["force"])
    energies = np.array(energies)
    forces = np.array(forces)

    block = TensorBlock(
        values=energies.reshape(-1, 1), 
        samples=Labels(["structure"], np.array([[i] for i in range(len(structures))], dtype=np.int32)), 
        components=[], 
        properties=Labels(["energy"], np.array([[0]], dtype=np.int32)),
    )

    block.add_gradient(
        "positions", 
        data=-forces.reshape(-1, 3, 1),
        samples=Labels(
            ["sample", "structure", "atom"], 
            np.array([
                [structure_i, structure_i, atom_i] 
                for structure_i, structure in enumerate(structures)
                for atom_i in range(len(structure))
            ], dtype=np.int32)
        ), 
        components=[Labels(["direction"], np.array([[0], [1], [2]], dtype=np.int32))]
    )
    
    return TensorMap(keys=Labels.single(), blocks=[block])


In [ ]:
train_energy = extract_energy_forces(train_structures)
test_energy = extract_energy_forces(test_structures)

We now have a TensorMap with a single block, containing both the energy for all 
structures and the forces acting on each atom.

In [ ]:
train_energy.block().samples

In [ ]:
train_energy.block().gradient("positions").samples[:10]

# feel free to explore the data more! in particular, try to understand what goes
# into the shape of train_energy.block().values and
# train_energy.block().gradient("positions").data

We will build a linear model using SOAP power spectrum features. The power
spectrum is built by taking the correlation of two spherical expansion, yielding
a 3-body invariant representation.

$$ \braket{\alpha \alpha' n n' l | \rho_i} = \sum_m \braket{\alpha n l m | \rho_i} \braket{\alpha' n' l m | \rho_i}$$

After computing the per-atom representation, we sum over the atoms in a
structure to create a per-structure representation:

$$ \braket{\alpha \alpha' n n' l | A} = \sum_i \braket{\alpha \alpha' n n' l | \rho_i}$$

In [ ]:
from utils.models import sum_over_structures

calculator = rascaline.SoapPowerSpectrum(
    cutoff=3.0, 
    max_radial=4,
    max_angular=4, 
    atomic_gaussian_width=0.3, 
    radial_basis={"Gto": {}}, 
    center_atom_weight=1.0, 
    gradients=True, 
    cutoff_function={"ShiftedCosine": {"width": 0.5}}
)

train_representation = calculator.compute(train_structures)
train_representation.keys_to_properties(["species_neighbor_1", "species_neighbor_2"])
train_representation.keys_to_samples("species_center")
train_representation = sum_over_structures(train_representation)

test_representation = calculator.compute(test_structures)
test_representation.keys_to_properties(["species_neighbor_1", "species_neighbor_2"])
test_representation.keys_to_samples("species_center")
test_representation = sum_over_structures(test_representation)

In [ ]:
def parity_plot(weights, train_energy, train_representation, test_energy, test_representation):
    fix, ax = plt.subplots(1, 2, figsize=(12, 6))

    train_reference = train_energy.block().values
    train_predicted = train_representation.block().values @ weights
    ax[0].scatter(train_reference.flatten(), train_predicted.flatten(), label="train")

    test_reference = test_energy.block().values
    test_predicted = test_representation.block().values @ weights
    ax[0].scatter(test_reference.flatten(), test_predicted.flatten(), label="test")

    ax[0].set_xlabel("reference")
    ax[0].set_ylabel("predicted")
    ax[0].set_title("energies")
    ax[0].legend()

    # Predicting the forces
    train_reference = train_energy.block().gradient("positions").data
    train_predicted = train_representation.block().gradient("positions").data @ weights
    ax[1].scatter(train_reference.flatten(), train_predicted.flatten(), label="train")

    test_reference = test_energy.block().gradient("positions").data
    test_predicted = test_representation.block().gradient("positions").data @ weights
    ax[1].scatter(test_reference.flatten(), test_predicted.flatten(), label="test")

    ax[1].set_xlabel("reference")
    ax[1].set_ylabel("predicted")
    ax[1].set_title("forces")
    ax[1].legend()

### Training a linear model on energies only

The first linear model we will make will only be trained on the energies. We can
use the ridge regression formula to compute the model weights $\beta$:

$$ \hat{y} = X \beta$$
$$ \beta = (X^T X + \Lambda)^{-1} X^T \ y$$

We can still use the gradients of the SOAP power spectrum to predict the forces
associated with this model; but since we did not include any force data when
training, the force predictions are really bad:

In [ ]:
X = train_representation.block().values
y = train_energy.block().values

regularizer = 1e-6

weights = np.linalg.solve(X.T @ X + regularizer * np.eye(X.shape[1]), X.T @ y)

parity_plot(weights, train_energy, train_representation, test_energy, test_representation)

### Training energies and all forces

We can improve the prediction on forces by including this data in the training set:

In [ ]:
X = train_representation.block().values
X_grad = train_representation.block().gradient("positions").data

y = train_energy.block().values
y_grad = train_energy.block().gradient("positions").data

X = np.vstack([X, X_grad.reshape(-1, X.shape[-1])])
y = np.vstack([y, y_grad.reshape(-1, y.shape[-1])])

regularizer = 1e-6

weights = np.linalg.solve(X.T @ X + regularizer * np.eye(X.shape[1]), X.T @ y)

parity_plot(weights, train_energy, train_representation, test_energy, test_representation)

Unfortunately, including all the forces on all the atoms increases the cost of
training a model quite a lot. Both the computational cost (CPU time) and memory
cost increase significantly. In this tutorial we are fine since we are working
with a very small dataset.

Let's compare training only on energies and training on energies and all forces:

In [ ]:
%%timeit

# Training on energies only

X = train_representation.block().values
y = train_energy.block().values

regularizer = 1e-6

weights = np.linalg.solve(X.T @ X + regularizer * np.eye(X.shape[1]), X.T @ y)

In [ ]:
%%timeit

# Training on forces and energies

X = train_representation.block().values
X_grad = train_representation.block().gradient("positions").data

y = train_energy.block().values
y_grad = train_energy.block().gradient("positions").data

X = np.vstack([X, X_grad.reshape(-1, X.shape[-1])])
y = np.vstack([y, y_grad.reshape(-1, y.shape[-1])])

regularizer = 1e-6

weights = np.linalg.solve(X.T @ X + regularizer * np.eye(X.shape[1]), X.T @ y)

### Training on energy and a subset of the forces

Equistore allows us to easily incorporate only a subset of the forces in our
model, still getting much better forces prediction, while reducing the
computational cost of training the model!

For example, let's use only the forces acting on the first hydrogen atom, for
the first 40 frames.

In [ ]:
X = train_representation.block().values
gradient = train_representation.block().gradient("positions")
# select the right representation entries
mask = np.logical_and(gradient.samples["atom"] == 1, gradient.samples["structure"] < 40)
X_grad = gradient.data[mask]

y = train_energy.block().values
gradient = train_energy.block().gradient("positions")
# select the right property entries
mask = np.logical_and(gradient.samples["atom"] == 1, gradient.samples["structure"] < 40)
y_grad = gradient.data[mask]

X = np.vstack([X, X_grad.reshape(-1, X.shape[-1])])
y = np.vstack([y, y_grad.reshape(-1, y.shape[-1])])

regularizer = 1e-6

weights = np.linalg.solve(X.T @ X + regularizer * np.eye(X.shape[1]), X.T @ y)

parity_plot(weights, train_energy, train_representation, test_energy, test_representation)

This model should be faster and require less memory to train!

In [ ]:
%%timeit

X = train_representation.block().values
gradient = train_representation.block().gradient("positions")
# select the right representation rows
mask = np.logical_and(gradient.samples["atom"] == 1, gradient.samples["structure"] < 40)
X_grad = gradient.data[mask]

y = train_energy.block().values
gradient = train_energy.block().gradient("positions")
# select the right property rows
mask = np.logical_and(gradient.samples["atom"] == 1, gradient.samples["structure"] < 40)
y_grad = gradient.data[mask]

X = np.vstack([X, X_grad.reshape(-1, X.shape[-1])])
y = np.vstack([y, y_grad.reshape(-1, y.shape[-1])])

regularizer = 1e-6

weights = np.linalg.solve(X.T @ X + regularizer * np.eye(X.shape[1]), X.T @ y)